In [1]:
from scipy import stats 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Epydemix import
import sys 
sys.path.append("../")
from epydemix.epimodel import EpiModel, simulate
from epydemix.plotting import plot_error_distribution, plot_selected_quantiles, plot_posterior, plot_posterior_2d
from epydemix.calibration import calibration_top_perc, calibration_abc_smc_old, calibration_abc_smc
from epydemix.utils import compute_simulation_dates

# Calibration

```epydemix``` currently supports three calibration methods: 
- ```calibration_top_perc```: this methods run a number of simulations and retain the top x%. This is the simpliest method, that allows to get results in the minimum amount of time, since the total number of simulations is set by the user
- ```calibration_rejection``` (not yet implemented): this method implement the simple rejection algorithm.
- ```calibration_abc_smc```: this is the most sophisticated method which implements the ABC-SMC algorithm. This method allows to achieve more accurate results, but it is more computationally expensive.

We import epidemiological data on which we want to calibrate the model. In particular we have two dataset: 
- data_I.csv: here the quantity of intereset is the number of individuals infectious at each time step (prevalence)

- data_incidence.csv: here the quantity of intereset is the number of new individuals infectious at each time step (incidence)

We use these two examples to show the flexibility of pur package to different type of data for calibration.

In [2]:
df_I = pd.read_csv("./data/data_I.csv")
df_incidence = pd.read_csv("./data/data_incidence.csv")

We first define model and priors, this step is common to all calibration methods:

In [3]:
# create model 
model = EpiModel(compartments=["S", "I", "R"], population_data_path="../epydemix_data/Indonesia/", population_name="Indonesia")
model.add_parameter(parameters_dict={"beta": 0.04, "mu": 0.2})
model.add_transition(source="S", target="I", rate="beta", agent="I")
model.add_transition(source="I", target="R", rate="mu")

# initial conditions
S = model.population.Nk - np.ones(len(model.population.Nk))
I = np.ones(len(model.population.Nk))

# simulation dates and compute contacts 
simulation_dates = compute_simulation_dates(start_date=df_I.date.values[0], end_date=df_I.date.values[-1], steps="daily")

# parameters
parameters = {"S": S, 
              "I": I, 
              "epimodel": model, 
              "simulation_dates": simulation_dates}

# priors
priors = {"beta": stats.uniform(0.015, 0.025), "mu": stats.uniform(0.15, 0.25)}

## Top % Calibration

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": results["I_total"]})
    return results

results = calibration_top_perc(simulate_wrapper, 
                               parameters=parameters,
                               priors=priors, 
                               data={"data": df_I["I"].values},
                               top_perc=0.05,
                               Nsim=1000)

In [ ]:
plot_selected_quantiles(results, ylabel="I_total")

In [ ]:
plot_posterior(results, "beta", kind="kde", prior_range=False, xlabel="$\\beta$")
plot_posterior(results, "mu", kind="kde", prior_range=False, xlabel="$\\mu$")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

In [ ]:
plot_error_distribution(results, kind="hist", xlabel="error metric")

### Incidence Example

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    return results

results = calibration_top_perc(simulation_function=simulate_wrapper, 
                               parameters=parameters,
                               priors=priors, 
                               data={"data": df_incidence["incidence"].values},
                               top_perc=0.05,
                               Nsim=1000)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

In [ ]:
plot_posterior(results, "beta", kind="kde", prior_range=False, xlabel="$\\beta$")
plot_posterior(results, "mu", kind="kde", prior_range=False, xlabel="$\\mu$")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

## ABC-SMC

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    return results

results = calibration_abc_smc_old(simulation_function=simulate_wrapper, 
                              priors=priors, 
                              parameters=parameters, 
                              data={"data": df_incidence["incidence"].values},  
                              population_size=50, 
                              max_nr_populations=4)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

In [ ]:
plot_posterior(results, "beta", kind="kde", prior_range=False, xlabel="$\\beta$")
plot_posterior(results, "mu", kind="kde", prior_range=False, xlabel="$\\mu$")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

In [ ]:
plot_error_distribution(results, kind="hist", xlabel="error metric")

# Custom ABC-SMC

In [4]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    return results

results = calibration_abc_smc(model=simulate_wrapper, 
                              priors=priors, 
                              parameters=parameters, 
                              observed_data={"data": df_incidence["incidence"].values},
                              num_particles=50, 
                              max_generations=4)

Generation 1: Tolerance = 2856300.8140684105, Accepted Particles = 50/50, Time = 0:00:03.966796, ESS = 46.813709177944816
Generation 2: Tolerance = 2498007.7420067554, Accepted Particles = 50/93, Time = 0:00:06.867247, ESS = 45.27183031292699
Generation 3: Tolerance = 1104378.7008304512, Accepted Particles = 50/168, Time = 0:00:12.418074, ESS = 48.84367951387517
Generation 4: Tolerance = 743320.8966358623, Accepted Particles = 50/131, Time = 0:00:09.652896, ESS = 48.63033776095266


In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

In [ ]:
def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    results["data"] = np.roll(results["data"], parameters["Delta"])
    results["data"][:10] = 0
    return results

priors = {"beta": stats.uniform(0.015, 0.025), "mu": stats.uniform(0.15, 0.25), "Delta": stats.randint(0, 10)}
results = calibration_abc_smc(model=simulate_wrapper, 
                              priors=priors, 
                              parameters=parameters, 
                              observed_data={"data": df_incidence["incidence"].values},
                              num_particles=50, 
                              max_generations=4)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

In [ ]:
plot_posterior_2d(results, "beta", "mu", kind="kde", prior_range=False, xlabel="$\\beta$", ylabel="$\\mu$")

In [ ]:
plot_posterior(results, "Delta", kind="kde", prior_range=False, xlabel="$\\Delta$") 

# Custom Initial Conditions 

In [ ]:
def custom_initial_conditions(epimodel, i0_perc): 
    initial_infected = int(np.sum(epimodel.population.Nk) * i0_perc)
    infected_by_age = np.random.multinomial(initial_infected, epimodel.population.Nk / epimodel.population.Nk.sum())
    return {"S": epimodel.population.Nk - infected_by_age, 
            "I": infected_by_age}

def simulate_wrapper(parameters): 

    # compute custom initial conditions 
    initial_conditions = custom_initial_conditions(parameters["epimodel"], parameters["i0_perc"])
    parameters.update(initial_conditions)
    
    results = simulate(**parameters)
    results.update({"data": np.concatenate(([0], (np.abs(np.diff(results["S_total"])))))})
    return results


results = calibration_abc_smc(simulation_function=simulate_wrapper, 
                              priors={"beta": stats.uniform(0.015, 0.025), 
                                      "mu": stats.uniform(0.15, 0.25), 
                                      "i0_perc": stats.uniform(0.0001, 0.01)}, 
                              parameters=parameters, 
                              data={"data": df_incidence["incidence"].values},  
                              population_size=50, 
                              max_nr_populations=4)

In [ ]:
plot_selected_quantiles(results, ylabel="I_total")

In [ ]:
plot_posterior(results, "i0_perc", kind="kde", prior_range=False, xlabel="$I_0$ (%)")

# Calibrating your own model

In [ ]:
from models.custom_models import my_stochastic_SIR

parameters = {
    'S': 273523000 - 10,
    'I': 10,
    'R': 0,
    'N': 273523000,
    'simulation_dates': simulation_dates,
    'dt': 1.0}

# priors
priors = {"beta": stats.uniform(0.15, 0.35), "mu": stats.uniform(0.15, 0.15)}

In [ ]:
results = calibration_top_perc(simulation_function=my_stochastic_SIR, 
                               parameters=parameters,
                               priors=priors, 
                               data={"data": df_incidence["incidence"].values},
                               top_perc=0.05,
                               Nsim=10000)

In [ ]:
plot_selected_quantiles(results, ylabel="incidence")

# Running Projections

In [ ]:
import pickle as pkl 
from epydemix.calibration import run_projections

def simulate_wrapper(parameters): 
    results = simulate(**parameters)
    results.update({"data": results["I_total"]})
    return results


with open("./test_calibration_results.pkl", "rb") as file: 
    results = pkl.load(file)


simulation_dates_projections = compute_simulation_dates(start_date=df_I.date.values[0], end_date=pd.to_datetime(df_I.date.values[-1]) + timedelta(days=28), steps="daily")
parameters_proj = parameters.copy()
parameters_proj["simulation_dates"] = simulation_dates_projections

projections = run_projections(simulation_function=simulate_wrapper, 
                              calibration_results=results, 
                              parameters=parameters_proj, 
                              iterations=100)

In [ ]:
from epydemix.plotting import get_timeseries_data
import seaborn as sns



In [ ]:
plot_projections(projections, results)

In [ ]:
plot_selected_quantiles(results)

In [ ]:
import pyabc

model = pyabc.ABCSMC().run()

In [13]:
import datetime
start = datetime.datetime.now()
end = datetime.datetime.now()

diff = end - start  


0:00:00.000027


In [19]:
# Format the difference as HH:MM:SS
hours, remainder = divmod(diff.total_seconds(), 3600)
minutes, seconds = divmod(remainder, 60)

formatted_time = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

print(f"Time difference: {formatted_time}")

Time difference: 00:00:00


In [18]:
diff.

AttributeError: 'datetime.timedelta' object has no attribute 'hour'